In [1]:
# It seems that JLD2 package is not present in jupyter server?
import Pkg;
Pkg.add("JLD2")

    Updating registry at `/opt/julia/registries/General`
   Resolving package versions...
  No Changes to `/opt/julia/environments/v1.6/Project.toml`
  No Changes to `/opt/julia/environments/v1.6/Manifest.toml`


In [3]:
using JuMP               # To write the optimisation models
using Cbc, Clp           # Solvers 
using Test               # Testing package
using JLD2;              # File I/O

# Homework 4

## Problem description

We need to fulfil the demand of clients using different servers. The demand and set of clients are unknown when making the decision on which servers to use. Consider the parameters:
- $C_j$ - cost of installing server $j$
- $P_s$ - probability of scenario $s$
- $F$ - cost of unmet demand (same for all clients $i$, servers $j$ and scenarios $s$)
- $Q_{ij}$ - benefit per one unit of demand of client $i$ served by server $j$
- $V$ - maximum allowed number of servers
- $D_{is}$ - demand of client $i$ in scenario $s$
- $U$ - maximum capacity of a server (same for all servers and scenarios)
- $H_{is}$ - a binary variable with value 1 if client $i$ is active in scenario $s$, i.e., the demand $D_{is}$ has to be fulfilled

Let the variables be

- $x_j$ - binary variable with value 1 if server $j$ is made available, i.e., built or installed
- $y_{ijs}$ - the proportion of demand $D_{is}$ fulfilled by server $j$. The total demand of $i$ served by $j$ is thus $y_{ijs} \times D_{is}$
- $z_{js}$ - capacity shortage for server $j$. If demand is not met otherwise, any server $j$ can procure emergency capacity at a price $F$.


The model is then given by:

\begin{align}
    \min_{x_j, z_{js}, y_{ijs}} & \sum_{j \in J} C_j x_j + \sum_{s} P_s \left( - \sum_{i \in I,j \in J}Q_{ij}D_{is}y_{ijs} + \sum_{j \in J} Fz_{js} \right) \\
    \text{s.t.: } & \sum_{j \in J} x_j \leq V   & (t)\\
    & \sum_{i \in I} D_{is}y_{ijs} - z_{js} \leq Ux_j, \forall j \in J, s \in S  & (u_{js})\\
    & \sum_{j \in J} y_{ijs} = H_{is}, \forall i \in I, s \in S  & (v_{is}) \\
    & x_j \in \{0,1\}, \ \forall j \in J \\
    & y_{ijs} \geq 0, \ \forall i \in I, \forall j \in J, \forall s \in S \\
    & z_{js} \geq 0, \ \forall j \in J, \forall s \in S
\end{align}

, where $t$, $u_{js}$, and $v_{is}$ are the dual variables related to the constraints in the model.

In [4]:
struct Instance
    # sets
    I  # Set of clients
    J  # Set of servers
    S  # Set of scenarios
    # Parameters 
    V  # Max number of servers
    P  # Scenario probabilities
    H  # 1 if client requires service
    C  # Cost of locating server at j
    F  # Cost of unmet demand
    D  # Demand in location i served from server j
    Q  # Benefit per unit of demand served
    U  # Maximum server capacity (same for all servers)
    loc_i # Coordinates of clients
    loc_j # Coordinates of servers
end

In [5]:
f = jldopen("hw4_ins.jld2")
ins = nothing
try
    ins = f["ins"]
finally
    close(f)
end;

# Task 1: implementing the large-scale model

## Model construction 

In the following, the full model must be implemented and solved using Cbc.

In [6]:
function generate_full_problem(ins)
    I = ins.I 
    J = ins.J
    S = ins.S
    V = ins.V 
    P = ins.P
    H = ins.H
    C = ins.C
    F = ins.F
    D = ins.D
    Q = ins.Q
    U = ins.U
    # Write the full model in JuMP

    # Initialize model
    m = Model(Cbc.Optimizer)
    
    # TODO: add your code here
    @variable(m, x[j=J], Bin)
    @variable(m, y[i=I,j=J,s=S] >= 0)
    @variable(m, z[j=J, s=S] >= 0)
    
    @objective(m, Min,
            sum(C[j]*x[j] for j in J) +
            F * sum(P[s] * z[j,s] for s in S, j in J) -
            sum(P[s] * Q[i,j] * D[i,s] * y[i,j,s] for s in S, i in I, j in J))
    
    @constraint(m, sum(x[j] for j in J) <= V)
    @constraint(m, [j in J, s in S], sum(D[i,s] * y[i,j,s] for i in I) <= U * x[j] + z[j,s])
    @constraint(m, [i in I, s in S], sum(y[i,j,s] for j in J) <= H[i,s])
    # Return the generated model
    return m
end

generate_full_problem (generic function with 1 method)

In [7]:
fullmodel = generate_full_problem(ins)
# set_silent(fullmodel)
optimize!(fullmodel)

Welcome to the CBC MILP Solver 
Version: 2.10.5 
Build Date: Jan  1 1970 

command line - Cbc_C_Interface -solve -quit (default strategy 1)
Continuous objective value is -1389.26 - 0.20 seconds
Cgl0004I processed model has 2900 rows, 12500 columns (5 integer (5 of which binary)) and 24995 elements
Cbc0038I Initial state - 5 integers unsatisfied sum - 1.36429
Cbc0038I Pass   1: suminf.    0.00000 (0) obj. 6024.31 iterations 1953
Cbc0038I Solution found of 6024.31
Cbc0038I Relaxing continuous gives 0
Cbc0038I Before mini branch and bound, 0 integers at bound fixed and 10398 continuous
Cbc0038I Full problem 2900 rows 12500 columns, reduced to 698 rows 2094 columns
Cbc0038I Mini branch and bound did not improve solution (0.41 seconds)
Cbc0038I Round again with cutoff of -138.893
Cbc0038I Pass   2: suminf.    0.10365 (2) obj. -138.893 iterations 1623
Cbc0038I Pass   3: suminf.    0.00000 (0) obj. -138.893 iterations 409
Cbc0038I Solution found of -138.893
Cbc0038I Relaxing continuous gives 

In [8]:
# Examine the solutions
@show x_bar = Int.(round.(value.(fullmodel[:x]).data))
@show opt_z = sum(value.(fullmodel[:z]))
@show objective_value(fullmodel);

x_bar = Int.(round.(value.(fullmodel[:x]).data)) = [0, 0, 0, 1, 1]
opt_z = sum(value.(fullmodel[:z])) = 0.0
objective_value(fullmodel) = -1327.366449586151


# Task 2: Benders decomposition

## Benders main

Formulate the initial main problem for the decomposition. Use a single variable $\theta$ for representing the subproblem value.

In [9]:
## Benders decomposition

## Generates the main problem
function generate_main(ins)
    
    J = ins.J
    V = ins.V
    C = ins.C
     
    main = Model(Cbc.Optimizer)
    set_silent(main)
    
    # TODO: add your code here
    @variable(main, x[j=J], Bin)
    @variable(main, θ)
    
    @objective(main, Min, sum(C[j]*x[j] for j in J) + θ)
    
    @constraint(main, sum(x[j] for j in J) <= V)
    return main  
end

generate_main (generic function with 1 method)

In [10]:
# Solve the main problem
function solve_main(ins, main)
    optimize!(main)
    return value.(main[:x]), value(main[:θ]), objective_value(main)    
end

solve_main (generic function with 1 method)

## Subproblem

Formulate the primal subproblem with corresponding objective value represented by the variable $\theta$ in the main problem. The primal subproblem is not used in the decomposition algorithm, but you will use it to test your implementation of the dual subproblem. It might also be easier to start by formulating the primal problem and then work from there to obtain the its dual formulation.

In [11]:
# Generate and solve the primal subproblem for a given x_bar. For test purposes only; if the dual is correct, the objective value of
# the dual subproblem must be the same as this.
function generate_and_solve_primal_subproblem(ins, x_bar)
    
    I = ins.I
    J = ins.J
    S = ins.S
    D = ins.D
    P = ins.P
    Q = ins.Q
    F = ins.F
    U = ins.U
    H = ins.H
    
    # set_silent works for Clp, and the subproblem should be an LP problem    
    sub = Model(Clp.Optimizer)
    set_silent(sub)
    
    # TODO: add your code here
    @variable(sub, y[i=I,j=J,s=S] >= 0)
    @variable(sub, z[j=J, s=S] >= 0)
    
    @objective(sub, Min, F * sum(P[s] * z[j,s] for s in S, j in J) - sum(P[s] * Q[i,j] * D[i,s] * y[i,j,s] for s in S, i in I, j in J))
    
    @constraint(sub, [j in J, s in S], sum(D[i,s] * y[i,j,s] for i in I) - z[j,s] <= U * x_bar[j])
    @constraint(sub, [i in I, s in S], sum(y[i,j,s] for j in J) == H[i,s])
    optimize!(sub)
    return objective_value(sub)
    
end

generate_and_solve_primal_subproblem (generic function with 1 method)

In [12]:
obj = generate_and_solve_primal_subproblem(ins, x_bar)
@show obj;

obj = -1460.3664495861508


## Dual subproblem

Formulate the dual subproblem. Consider the dual variables indicated in the fullmodel as we are expecting you to use the same names. Hint: You can find the conversion rules for primal and dual problems in Lecture 5.

In [13]:
## Define Benders subproblem
function generate_and_solve_dual_subproblem(ins, x_bar)
    
    I = ins.I
    J = ins.J
    S = ins.S
    D = ins.D
    P = ins.P
    Q = ins.Q
    F = ins.F
    U = ins.U
    H = ins.H
    
    # set_silent works for Clp, and the subproblem should be an LP problem
    sub_dual = Model(Clp.Optimizer)
    set_silent(sub_dual)
    
    # TODO: add your code here
    @variable(sub_dual, v[i=I, s=S])
    @variable(sub_dual, u[j=J, s=S] <= 0)
    
    @objective(sub_dual, Max, sum(H[i,s]*v[i,s] for s in S, i in I) + sum(U*x_bar[j]*u[j,s] for s in S, j in J))
    
    @constraint(sub_dual, [i in I, j in J, s in S], D[i,s]*u[j,s] + v[i,s] <= -P[s]*D[i,s]*Q[i,j])
    @constraint(sub_dual, [j in J, s in S], -u[j,s] <= P[s]*F)

    optimize!(sub_dual)
    
    u_bar = value.(sub_dual[:u])                     
    v_bar = value.(sub_dual[:v])                     
    opt_value = objective_value(sub_dual)
    
    return u_bar, v_bar, opt_value
end

generate_and_solve_dual_subproblem (generic function with 1 method)

## Benders cut

Formulate the Benders optimality cut. Remember to explain in your report why you only need to consider one type of cut.

In [14]:
# Add the Benders cut, given current dual values
function add_benders_cut(ins, main, u_bar, v_bar)   
    
    U = ins.U
    H = ins.H
    I = ins.I
    J = ins.J
    S = ins.S
    
    x = main[:x]
    θ = main[:θ]
    
    @constraint(main, sum(H[i,s]*v_bar[i,s] for s in S, i in I) + sum(U*x[j]*u_bar[j,s] for s in S, j in J) <= θ
    # TODO: add your code here
    )
    return main
end

add_benders_cut (generic function with 1 method)

### Testing the subproblem formulation

You can use the cell below to check whether your implementation of the subproblem is correct. For a fixed solution from the main problem, strong duality holds and thus these objective function values should match. We use `≈` which is equivalent to `approx()` to test whether the values are sufficiently close.

In [15]:
## Test that the primal and dual solutions are the same
(u,v,optval) = generate_and_solve_dual_subproblem(ins, x_bar)
obj = generate_and_solve_primal_subproblem(ins, x_bar)
@test optval ≈ obj

Test Passed

## Benders decomposition algorithm

Here you will combine the functions you defined before into the complete algorithm. 

Some hints:
- You should add a cut before solving the main problem for the first time to make the problem bounded (in the initialisation of the algorithm).
- For the single cut problem, you can ignore the indices $k$ on the lecture slides, as there is only one subproblem being solved.

In [16]:
function benders_decomposition(ins; max_iter = 100)
    
    k = 1
    ϵ = 0.01
    LB = -Inf
    UB = +Inf
    gap = +Inf
    x_bar = zeros(length(ins.J))
    
    start = time()
    
    # TODO: initialize the main problem and add one Benders cut to make the problem bounded
    main = generate_main(ins)
    (u_bar,v_bar,optval) = generate_and_solve_dual_subproblem(ins, x_bar)
    main = add_benders_cut(ins, main, u_bar, v_bar)
    while k <= max_iter && gap > ϵ
        # TODO: obtain necessary solutions
        (x_bar, θ_bar, obj_val_main) = solve_main(ins, main)
        (u_bar, v_bar, obj_val_SD) = generate_and_solve_dual_subproblem(ins, x_bar)
        
        LB = obj_val_main # TODO: what is the lower bound for the objective?
        UB = min(UB, obj_val_main - θ_bar + obj_val_SD) # TODO: what about the upper bound?
        gap = abs((UB - LB) / UB)
        println("Iter $(k): UB: $(UB), LB: $(LB), gap $(gap)")
        
        if gap <= ϵ # Lower and upper bounds are (practically) same and the solution is thus optimal
            stop = time()
            println("Optimal found. \n Objective value: $(round(UB, digits=2)). \n Total time: $(round(stop-start, digits=2))s")
            return
        else
            # TODO: optimality not reached, modify the main problem for the next iteration
            main = add_benders_cut(ins, main, u_bar, v_bar)
            k += 1
            end
        end
    println("Maximum number of iterations exceeded")
    end

benders_decomposition (generic function with 1 method)

In [17]:
benders_decomposition(ins)

Iter 1: UB: -1263.0273793846552, LB: -63877.147464181195, gap 49.57463401569492
Iter 2: UB: -1263.0273793846552, LB: -63695.02737938466, gap 49.43044071650827
Iter 3: UB: -1299.6094936117247, LB: -32539.027379384654, gap 24.03754207654789
Iter 4: UB: -1303.8651121860437, LB: -32529.027379384654, gap 23.948153820027365
Iter 5: UB: -1303.8651121860437, LB: -32523.60949361173, gap 23.94399856982373
Iter 6: UB: -1303.8651121860437, LB: -32515.60949361173, gap 23.937862965822035
Iter 7: UB: -1303.8651121860437, LB: -32473.865112186046, gap 23.905847091606567
Iter 8: UB: -1303.8651121860437, LB: -32473.51268758718, gap 23.90557679938418
Iter 9: UB: -1303.8651121860437, LB: -32469.60949361173, gap 23.902583242812284
Iter 10: UB: -1303.8651121860437, LB: -32462.36977215019, gap 23.89703073481596
Iter 11: UB: -1303.8651121860437, LB: -32459.60949361173, gap 23.894913737810164
Iter 12: UB: -1303.8651121860437, LB: -32452.36977215019, gap 23.88936122981384
Iter 13: UB: -1303.8651121860437, LB: -3

In [18]:
objective_value(fullmodel)

-1327.366449586151

# Task 3: 

## Benders components (multi-cut version)

Your task is to create a version of the main problem with multiple Benders cuts being generated at each iteration, and the respective Benders cut. We refer to this version as the multi-cut version. 

Here is a bonus question that might give you ideas on how the implementation could be made more efficient: notice that the previous implementation of the dual subproblem is generating all the cut information at once, and that is why we can reutilise the function `solve_dual_subproblem(ins, x_bar)` here. Imagining that you have a number of parallel computing nodes available, can you see a way that the function `solve_dual_subproblem(ins, x_bar)` could be made more efficient? (bear in mind you are **not required** to implement or try anything in the direction of the answer to the bonus question, but only to give the question a thought!)

In [19]:
## Benders decomposition: multi-cut

## Generates the main problem
function generate_main_multi(ins)
    
    J = ins.J
    S = ins.S
    V = ins.V
    C = ins.C
    
    main = Model(Cbc.Optimizer)
    set_silent(main)
    
    # TODO: add your code here
    @variable(main, x[j=J], Bin)
    @variable(main, θ[s=S])
    
    @objective(main, Min, sum(C[j]*x[j] for j in J) + sum(θ[s] for s in S))
    
    @constraint(main, sum(x[j] for j in J) <= V)
    return main  
end

# Solve the main problem
function solve_main_multi(ins, main)
    
    optimize!(main)
    
    return value.(main[:x]), value.(main[:θ]), objective_value(main)    

end


# Add the Benders cut, given current dual values
function add_benders_cut_multi(ins, main, u_bar, v_bar)   
    
    U = ins.U
    H = ins.H
    I = ins.I
    J = ins.J
    S = ins.S

    x = main[:x]
    θ = main[:θ]
    
    @constraint(main, [s in S], sum(H[i,s]*v_bar[i,s] for i in I) + sum(U*x[j]*u_bar[j,s] for j in J) <= θ[s]
    # TODO: add your code here
    )

    return main
end

add_benders_cut_multi (generic function with 1 method)

In [20]:
function benders_decomposition_multi(ins; max_iter = 100)
    
    k = 1
    ϵ = 0.01
    LB = -Inf
    UB = +Inf
    gap = +Inf
    x_bar = zeros(length(ins.J))
    
    start = time()
    
    # TODO: initialize the main problem and add one set of Benders cuts to make the problem bounded
    main = generate_main_multi(ins)
    (u_bar,v_bar,optval) = generate_and_solve_dual_subproblem(ins, x_bar)
    main = add_benders_cut_multi(ins, main, u_bar, v_bar)
    while k <= max_iter && gap > ϵ
        # TODO: obtain necessary solutions
        (x_bar, θ_bar, obj_val_main) = solve_main_multi(ins, main)
        (u_bar, v_bar, obj_val_SD) = generate_and_solve_dual_subproblem(ins, x_bar)
        
        LB = obj_val_main # TODO: what is the lower bound for the objective?
        UB = min(UB, obj_val_main - sum(θ_bar[s] for s in ins.S) + obj_val_SD)# TODO: what about the upper bound?
        
        gap = abs((UB - LB) / UB)
        println("Iter $(k): UB: $(UB), LB: $(LB), gap $(gap)")
        
        if gap <= ϵ # Lower and upper bounds are (practically) same and the solution is thus optimal
            stop = time()
            println("Optimal found. \n Objective value: $(round(UB, digits=2)). \n Total time: $(round(stop-start, digits=2))s")
            return
        else
            # TODO: optimality not reached, modify the main problem for the next iteration
            main = add_benders_cut_multi(ins, main, u_bar, v_bar)
            k += 1
            end
        end
    println("Maximum number of iterations exceeded")
    end

benders_decomposition_multi (generic function with 1 method)

In [21]:
benders_decomposition_multi(ins)

Iter 1: UB: -1263.0273793846625, LB: -63877.14746418113, gap 49.57463401569458
Iter 2: UB: -1263.0273793846625, LB: -61701.068317767415, gap 47.85172667264562
Iter 3: UB: -1263.0273793846625, LB: -32483.24397196351, gap 24.71855883899295
Iter 4: UB: -1263.0273793846625, LB: -32479.933919287298, gap 24.715938109838348
Iter 5: UB: -1273.1081135244863, LB: -32471.933919287298, gap 24.50603014334081
Iter 6: UB: -1278.2087360342412, LB: -32470.86182525683, gap 24.403410968697205
Iter 7: UB: -1278.2087360342412, LB: -32469.755129527006, gap 24.402545151011388
Iter 8: UB: -1278.2087360342412, LB: -32463.10586476371, gap 24.39734313308126
Iter 9: UB: -1278.2087360342412, LB: -32454.341405397172, gap 24.390486303582712
Iter 10: UB: -1278.2087360342412, LB: -32453.245446661687, gap 24.38962888592894
Iter 11: UB: -1327.3664495861508, LB: -1329.9440775985843, gap 0.0019419113789090703
Optimal found. 
 Objective value: -1327.37. 
 Total time: 5.86s
